In [12]:
import re
import copy
import pydot_ng as pydot
class Morph:
    def __init__(self,item):
        self.surface=item[0]
        self.base=item[len(items)-3]
        self.pos=items[1]
        self.pos1=items[2] 
        
class Chunk:
    def __init__(self,idx,dst):
        self.idx = idx     
        self.morphs = []   
        self.dst = dst     
        self.srcs = [] 
        
        
with open("ai.ja.txt.parsed","r",encoding="utf-8") as ai:
    result=[]
    sent=[]
    for line in ai:
        line=line.strip()
        items=re.split("[ ,\t]",line)
        if items[0]=="*":
            idx=int(items[1])
            items[2]=items[2].strip("D")
            dst=int(items[2])
            chunk=Chunk(idx,dst)
            sent.append(chunk)
            
        elif items[0]=="EOS":
            if sent:
                result.append(sent)
                sent=[]
            
        else:
            chunk.morphs.append(Morph(items))
            
    for sen in result:
        for chunk in sen:
            if chunk.dst!=-1:
                sen[int(chunk.dst)].srcs.append(chunk.idx)
    
#49
    root=[]
    roots=[]
    roots_sen=[]
    root_pos=[]
    roots_pos=[]
    roots_pos_sen=[]
    for sent in result:
        for chunk in sent:
            if chunk.dst!=-1:
                moto_sur=[k.surface if k.pos!="記号" else "" for k in chunk.morphs]
                moto_sur_pos=[k.pos if k.pos!="記号" else "" for k in chunk.morphs]
                root.append(moto_sur)
                root_pos.append(moto_sur_pos)
                next_dst=chunk.dst
                
                while next_dst!=-1:
                    next=[k.surface if k.pos!="記号" else "" for k in sent[next_dst].morphs]
                    next_pos=[k.pos if k.pos!="記号" else "" for k in sent[next_dst].morphs]
                    root.append(next)
                    root_pos.append(next_pos)
                    next_dst=sent[int(next_dst)].dst
            roots.append(root)
            roots_pos.append(root_pos)
            root=[]
            root_pos=[]
        roots_sen.append(roots)
        roots_pos_sen.append(roots_pos)
        roots=[]
        roots_pos=[]
     
    kumiawase=1
    base=[]
    base_pos=[]
    search=[]
    search_pos=[]
    
    base_xy=[]
    for items,pos_s in zip(roots_sen[2],roots_pos_sen[2]):
        if len(items)>=2 and kumiawase!=len(roots_sen[2]):
            word_num=1
            print(kumiawase,": ",end="")
            for w in items:
                if "" in w:
                    w.remove("")
                base.append(w)
                print("".join(w),end="")
                if len(items)-1>=word_num:
                    print("->",end="")
                    word_num+=1
                    
            for p in pos_s:
                if "" in p:
                    p.remove("")
                base_pos.append(p)
            print("\n")
            
            base2=copy.deepcopy(base)
            for n in range(2,len(base2)):#moto_bunkai
                base2=copy.deepcopy(base)
                for k in range(0,int(n)):
                    k1=0
                    while base_pos[k][k1]=="名詞":
                        base2[k][k1]=""
                        k1+=1
                    if k1>0:
                        if k==0:
                            base2[k][k1-1]="x"
                        elif k==int(n)-1:
                            base2[k][k1-1]="y"
                    print("".join(base2[k]),end="")
                    if k+1!=int(n):
                        print("->",end="")
                print("\n")
            
            for i in range(int(kumiawase),len(roots_sen[2])-1):
                itti=0
                for i2 in range(len(roots_sen[2][i])):
                    if "" in roots_sen[2][i][i2]:
                        roots_sen[2][i][i2].remove("")
                        roots_pos_sen[2][i][i2].remove("")
                    search.append(roots_sen[2][i][i2])
                    search_pos.append(roots_pos_sen[2][i][i2])

                for num in range(len(search)-1,-1,-1) :
                    if(search[num]==base[len(base)-1-itti]):
                        itti+=1 #何件一致してるか
                    else:
                        break        
                count=0
                while base_pos[0][count]=="名詞":
                    base[0][count]=""
                    count+=1
                if count>=1:
                    base[0][count-1]="x"
                 
                
                
                for num2 in range(0,len(base)-itti):
                    if num2!=len(base)-itti and int(num2)!=0:
                        print("->",end="")
                    print("".join(base[num2]),end="")
                    
                print(" | ",end="")
                
                for num3 in range(0,len(search)):
                    if num3==len(search)-itti:
                        print(" | ",end="")
                        print("".join(search[num3]),end="")
                    elif num3>=len(search)-itti:
                        print("".join(search[num3]),end="")
                    else:
                        if num3==0:
                            m=0
                            while search_pos[num3][m]=="名詞":
                                m+=1
                            print("y",end="")
                            for m2 in range(m,len(search_pos[num3])):
                                print(search[num3][m2],end="")
                        else:
                            if num3==1:
                                print("->",end="")
                            print("".join(search[num3]),end="")
                            if num3+itti+1!=len(search):
                                print("->",end="")
                
                search=[]
                search_pos=[]
                print()
           
            kumiawase+=1
            base=[]
            base_pos=[]
            print()
        
  

1 : 日本大百科全書(ニッポニカ)』の->解説で->述べている

x大百科全書(ニッポニカ)』の->yで

x大百科全書(ニッポニカ)』の |  | 解説で述べている
x大百科全書(ニッポニカ)』の->解説で | yの->佐藤理史は | 述べている
x大百科全書(ニッポニカ)』の->解説で | yは | 述べている
x大百科全書(ニッポニカ)』の->解説で | yのように | 述べている

2 : 解説で->述べている

xで | yの->佐藤理史は | 述べている
xで | yは | 述べている
xで | yのように | 述べている

3 : 情報工学者通信工学者の->佐藤理史は->述べている

xの->yは

xの |  | 佐藤理史は述べている
xの->佐藤理史は | yのように | 述べている

4 : 佐藤理史は->述べている

xは | yのように | 述べている

5 : 次のように->述べている


